In [1]:
from langgraph.graph import StateGraph, END
import random
from typing_extensions import TypedDict
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os
from typing import List, Dict, Any
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.json import JsonOutputParser


# Estado

In [2]:
class State(TypedDict):
    user_input: str
    modo: str
    respuestas: List[str]
    pregunta_idx: int
    feedback: Dict[str, Any]
    nivel: str
    fortalezas: List[str]
    debilidades: List[str]
    puntaje_promedio: float
    detalle: List[Dict[str, Any]]
    preguntas_seleccionadas: List[Dict[str, Any]]
    temas: List[str]
    subtemas : Dict[str, Any]
    tema_actual : int

# Preguntas Quiz

In [3]:
quiz_preguntas = {
    "basico": [
        {"tema": "Variables aleatorias", "pregunta": "¿Qué es una variable aleatoria?"},
        {"tema": "Variables aleatorias", "pregunta": "¿Cuál es la diferencia entre una variable aleatoria discreta y continua?"},
        {"tema": "Probabilidad", "pregunta": "¿Qué es la probabilidad clásica?"},
        {"tema": "Probabilidad", "pregunta": "¿Qué es la probabilidad frecuentista?"},
        {"tema": "Distribuciones simples", "pregunta": "¿Qué es una distribución uniforme?"},
        {"tema": "Distribuciones simples", "pregunta": "¿Qué caracteriza a una distribución de probabilidad discreta?"},
        {"tema": "Eventos", "pregunta": "¿Qué es un evento en probabilidad?"},
        {"tema": "Eventos", "pregunta": "¿Qué significa que dos eventos sean mutuamente excluyentes?"}
    ],
    "intermedio": [
        {"tema": "Desviación estándar", "pregunta": "¿Qué es la desviación estándar?"},
        {"tema": "Desviación estándar", "pregunta": "¿Cómo se interpreta una desviación estándar alta o baja?"},
        {"tema": "Medidas de dispersión", "pregunta": "¿Qué es la varianza?"},
        {"tema": "Medidas de dispersión", "pregunta": "¿Cómo se relacionan la varianza y la desviación estándar?"},
        {"tema": "Estadística descriptiva", "pregunta": "¿Qué es la media aritmética?"},
        {"tema": "Estadística descriptiva", "pregunta": "¿Qué diferencia hay entre la media y la mediana?"},
        {"tema": "Distribuciones", "pregunta": "¿Qué es una distribución normal?"},
        {"tema": "Distribuciones", "pregunta": "¿Qué es una distribución sesgada y cómo se identifica?"}
    ],
    "avanzado": [
        {"tema": "Probabilidad conjunta", "pregunta": "¿Cómo se calcula la probabilidad conjunta de dos eventos independientes?"},
        {"tema": "Probabilidad conjunta", "pregunta": "¿Qué diferencia hay entre probabilidad conjunta y probabilidad condicional?"},
        {"tema": "Teorema de Bayes", "pregunta": "Explica el teorema de Bayes con un ejemplo."},
        {"tema": "Teorema de Bayes", "pregunta": "¿Cómo se aplica el teorema de Bayes en problemas médicos?"},
        {"tema": "Distribuciones avanzadas", "pregunta": "¿Qué es una distribución binomial?"},
        {"tema": "Distribuciones avanzadas", "pregunta": "¿Qué es una distribución de Poisson y en qué casos se utiliza?"},
        {"tema": "Inferencia", "pregunta": "¿Qué es una estimación puntual en inferencia estadística?"},
        {"tema": "Inferencia", "pregunta": "¿Qué diferencia hay entre una estimación puntual y un intervalo de confianza?"}
    ]
}


# Configuracion LLM

In [4]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
llm = ChatOpenAI(model="o4-mini")

In [6]:
# prueba llm
respuesta = llm.invoke("¿Qué es una variable aleatoria?")
respuesta

AIMessage(content='En teoría de la probabilidad una **variable aleatoria** es, en términos sencillos, una regla (o función) que asigna un número real a cada resultado posible de un experimento aleatorio.  \n\n1. Definición formal  \n   Sea Ω el espacio muestral (el conjunto de todos los resultados posibles de un experimento). Una variable aleatoria X es una función  \n   X: Ω → ℝ  \n   de modo que a cada ω∈Ω (cada posible “suceso elemental”) le asigna un valor real X(ω).\n\n2. Por qué «aleatoria»  \n   El adjetivo “aleatoria” no significa que X cambie arbitrariamente, sino que antes de hacer el experimento no sabemos qué valor tomará; su valor queda determinado por el resultado del experimento.\n\n3. Tipos principales  \n   a) Discreta  \n      • Toma un conjunto contable de valores (por ejemplo {0,1,2,…}).  \n      • Se describe por su función de masa de probabilidad pX(k)=P(X=k).  \n      Ejemplo: lanzar un dado justo y definir X = número de puntos obtenido (X∈{1,2,3,4,5,6}).  \n   b

# Nodos

## nodo_inicio

In [22]:
## nodo
def classify_node(state):
    user_input = state["user_input"]
    if not user_input:
            return {"modo": ""}
    prompt = ChatPromptTemplate.from_messages([
        ("system",
        "Clasifica la siguiente intención del usuario SOLO como 'guiado' o 'libre'. "
        "Si el usuario quiere que lo guíes paso a paso, responde 'guiado'. "
        "Si solo quiere una respuesta directa, responde 'libre'. "
        "Intención del usuario: {user_input}\n"
        "Respuesta:")
    ])
    #respuesta = llm.invoke(prompt).content.strip().lower()
    llm_chain = prompt | llm
    respuesta = llm_chain.invoke({"user_input": user_input}).content.strip().lower()
    if "guiado" in respuesta:
        modo = "guiado"
    elif "libre" in respuesta:
        modo = "libre"
    else:
        modo = "libre"
    return {"modo": modo}

In [23]:
#edge
def transicion_inicio(state: State):
    if state.modo == "modo libre":
        return "quiz_nivel"
    else:
        return "libre"

In [24]:
estado_prueba = State(
    user_input="quiero que me guies paso a paso",
    modo="",
    respuestas=[],
    pregunta_idx=0,
    feedback={},
    nivel="basico",
    fortalezas=[],
    debilidades=[],
    puntaje_promedio=0.0,
    detalle=[],
    preguntas_seleccionadas=[]
)
classify_node(estado_prueba)

{'modo': 'guiado'}

## nodo quiz_nivel

In [81]:
prompt_quiz = ChatPromptTemplate.from_messages([
    ("system",
     """Eres un experto en educación. Evalúa las siguientes respuestas del usuario a preguntas de probabilidad y estadística.
Para cada respuesta, califica de 0 a 5 (donde 0 es incorrecta y 5 es perfecta), explica brevemente la calificación.

Devuelve la respuesta SOLO en formato JSON con la siguiente estructura:
{{
  "resultados": [puntaje1, puntaje2, ...],
  "detalle": [
    {{
      "pregunta": "...",
      "respuesta": "...",
      "tema": "...",
      "puntaje": 0-5,
      "feedback": "..."
    }},
    ...
  ]
}}

Respuestas del usuario:
{respuestas_usuario}
""")
])

In [ ]:
# nodo
def nodo_calificar(state):
    parser = JsonOutputParser()
    chain_quiz = prompt_quiz | llm | parser
    
    respuestas = state.get("respuestas", [])
    preguntas_seleccionadas = state.get("preguntas_seleccionadas", [])
    respuestas_usuario = []
    for idx, pregunta in enumerate(preguntas_seleccionadas):
        if idx < len(respuestas):
            respuestas_usuario.append({
                "pregunta": pregunta["pregunta"],
                "respuesta": respuestas[idx],
                "tema": pregunta["tema"]
            })
    prompt_str = prompt_quiz.format(respuestas_usuario=str(respuestas_usuario))
    data = chain_quiz.invoke(prompt_str)

    resultados = data.get("resultados", [])
    detalle = data.get("detalle", [])
    promedio = sum(resultados) / len(resultados) if resultados else 0
    promedio = round(promedio, 2)
    fortalezas = [d["tema"] for d in detalle if d["puntaje"] >= 4]
    debilidades = [d["tema"] for d in detalle if d["puntaje"] < 4]
    state["feedback"] = data
    state["fortalezas"] = list(set(state.get("fortalezas", []) + fortalezas))
    state["debilidades"] = list(set(state.get("debilidades", []) + debilidades))
    state["puntaje_promedio"] = promedio
    state["detalle"] = detalle
    return state

In [84]:
estado_prueba = State(
    user_input="quiero que me guies paso a paso",
    modo="modo guiado",
    # Lista de respuestas del usuario
    respuestas=["Python es un lenguaje de programación interpretado",
                "Un bucle for se usa para iterar sobre una secuencia",
                "Una variable es un espacio en memoria que almacena datos"],

    pregunta_idx=3,  # Indica que ya se respondieron 3 preguntas

    # Las preguntas que fueron seleccionadas y respondidas
    preguntas_seleccionadas=[
        {
            "pregunta": "¿Qué es Python?",
            "tema": "Fundamentos de Python",
            "respuesta_correcta": "Python es un lenguaje de programación interpretado de alto nivel"
        },
        {
            "pregunta": "¿Para qué se usa un bucle for?",
            "tema": "Estructuras de Control",
            "respuesta_correcta": "Un bucle for se utiliza para iterar sobre una secuencia de elementos"
        },
        {
            "pregunta": "¿Qué es una variable?",
            "tema": "Conceptos Básicos",
            "respuesta_correcta": "Una variable es un espacio en memoria que almacena un valor"
        }
    ],

    feedback={},  # Se llenará después de la calificación
    nivel="basico",
    fortalezas=[],  # Se llenará después de la calificación
    debilidades=[],  # Se llenará después de la calificación
    puntaje_promedio=0.0,  # Se calculará después de la calificación
    detalle=[]  # Se llenará después de la calificación
)

# Probar el nodo
estado_actualizado = nodo_calificar(estado_prueba)

# Imprimir los resultados actualizados
print("\nResultados actualizados:")
print(f"Puntaje promedio: {estado_actualizado['puntaje_promedio']}")
print(f"Fortalezas: {estado_actualizado['fortalezas']}")
print(f"Debilidades: {estado_actualizado['debilidades']}")
print(f"Detalle: {estado_actualizado['detalle']}")



Resultados actualizados:
Puntaje promedio: 5.0
Fortalezas: ['Fundamentos de Python', 'Conceptos Básicos', 'Estructuras de Control']
Debilidades: []
Detalle: [{'pregunta': '¿Qué es Python?', 'respuesta': 'Python es un lenguaje de programación interpretado', 'tema': 'Fundamentos de Python', 'puntaje': 5, 'feedback': 'Definición precisa y completa de Python como lenguaje interpretado.'}, {'pregunta': '¿Para qué se usa un bucle for?', 'respuesta': 'Un bucle for se usa para iterar sobre una secuencia', 'tema': 'Estructuras de Control', 'puntaje': 5, 'feedback': 'Explicación correcta y clara del propósito de un bucle for.'}, {'pregunta': '¿Qué es una variable?', 'respuesta': 'Una variable es un espacio en memoria que almacena datos', 'tema': 'Conceptos Básicos', 'puntaje': 5, 'feedback': 'Descripción exacta de una variable y su función en programación.'}]


## nodo plan_estudio

In [85]:
def nodo_plan_estudio(state: State):
    # Corregido: usar [] en lugar de .
    debilidades = state["debilidades"]
    debilidades_str = ", ".join(debilidades)
    print(f"Debilidades: {debilidades_str}")

    prompt_plan = ChatPromptTemplate.from_template(
        """
    Eres un tutor experto en estadística y probabilidad.
    El estudiante tiene el nivel: {nivel}.
    Sus debilidades principales son: {debilidades}.

    Crea un plan de estudio personalizado y devuélvelo SOLO en formato JSON con EXACTAMENTE esta estructura:
    {{
        "plan_estudio": {{
            "tema1": {{
                "nombre": "Nombre del Tema 1",
                "subtemas": [
                    "Subtema 1.1",
                    "Subtema 1.2",
                    "Subtema 1.3",
                    "Subtema 1.4"
                ]
            }},
            "tema2": {{
                "nombre": "Nombre del Tema 2",
                "subtemas": [
                    "Subtema 2.1",
                    "Subtema 2.2",
                    "Subtema 2.3",
                    "Subtema 2.4"
                ]
            }},
            "tema3": {{
                "nombre": "Nombre del Tema 3",
                "subtemas": [
                    "Subtema 3.1",
                    "Subtema 3.2",
                    "Subtema 3.3",
                    "Subtema 3.4"
                ]
            }}
        }}
    }}

    Los temas deben enfocarse en las debilidades mencionadas.
    Cada tema DEBE tener exactamente 4 subtemas.
    IMPORTANTE: Devuelve SOLO el JSON, sin texto adicional.
    """
    )

    parser = JsonOutputParser()
    chain = prompt_plan | llm | parser

    # Corregido: usar [] para acceder a los elementos
    respuesta = chain.invoke({
        "nivel": state["nivel"],
        "debilidades": debilidades_str
    })

    plan = respuesta["plan_estudio"]
    state_actualizado = state.copy()

    # Actualizar el estado con la información del plan
    temas = []
    subtemas = {}

    for tema_key, tema_data in plan.items():
        nombre_tema = tema_data["nombre"]
        temas.append(nombre_tema)
        subtemas[nombre_tema] = tema_data["subtemas"]

    state_actualizado["temas"] = temas
    state_actualizado["subtemas"] = subtemas
    state_actualizado["tema_actual"] = 0

    # Imprimir el plan de forma legible
    print("\nPlan de estudio personalizado:")
    for i, tema in enumerate(temas, 1):
        print(f"\n{i}. {tema}")
        for subtema in subtemas[tema]:
            print(f"   • {subtema}")

    return state_actualizado

In [86]:
# Ejemplo de uso
estado_inicial = State(
    user_input="",
    modo="guiado",
    respuestas=[],
    pregunta_idx=0,
    feedback={},
    nivel="intermedio",
    fortalezas=[],
    debilidades=["probabilidad condicional", "distribuciones", "teorema de Bayes"],
    puntaje_promedio=0.0,
    detalle=[],
    preguntas_seleccionadas=[],
    temas=[],
    subtemas={},
    tema_actual=0
)

In [87]:
nodo_plan_estudio(estado_inicial)

Debilidades: probabilidad condicional, distribuciones, teorema de Bayes

Plan de estudio personalizado:

1. Probabilidad Condicional
   • Definición y fórmula de P(A|B)
   • Regla de la multiplicación
   • Teorema de la probabilidad total
   • Ejemplos prácticos con árboles de probabilidad

2. Distribuciones de Probabilidad
   • Distribuciones discretas: Binomial y Poisson
   • Distribuciones continuas: Uniforme y Normal
   • Función de densidad y función de distribución acumulada
   • Cálculo de esperanza y varianza

3. Teorema de Bayes
   • Derivación de la fórmula de Bayes
   • Priori, verosimilitud y posteriori
   • Aplicaciones en diagnósticos médicos
   • Resolución de problemas complejos


{'user_input': '',
 'modo': 'guiado',
 'respuestas': [],
 'pregunta_idx': 0,
 'feedback': {},
 'nivel': 'intermedio',
 'fortalezas': [],
 'debilidades': ['probabilidad condicional',
  'distribuciones',
  'teorema de Bayes'],
 'puntaje_promedio': 0.0,
 'detalle': [],
 'preguntas_seleccionadas': [],
 'temas': ['Probabilidad Condicional',
  'Distribuciones de Probabilidad',
  'Teorema de Bayes'],
 'subtemas': {'Probabilidad Condicional': ['Definición y fórmula de P(A|B)',
   'Regla de la multiplicación',
   'Teorema de la probabilidad total',
   'Ejemplos prácticos con árboles de probabilidad'],
  'Distribuciones de Probabilidad': ['Distribuciones discretas: Binomial y Poisson',
   'Distribuciones continuas: Uniforme y Normal',
   'Función de densidad y función de distribución acumulada',
   'Cálculo de esperanza y varianza'],
  'Teorema de Bayes': ['Derivación de la fórmula de Bayes',
   'Priori, verosimilitud y posteriori',
   'Aplicaciones en diagnósticos médicos',
   'Resolución de pr

# Nodo explicacion (RAG)

## Rag dummy para prueba en UI

### Toca hacer que solo explique un tema que lo devuelva en formato mark down, y que luego la UI le vaya pasando 1 a 1 los temas segun avanza el ciclo

In [3]:
from typing import TypedDict, List, Dict, Any, Annotated
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage
from operator import itemgetter
from langchain_chroma import Chroma  # Importación actualizada
from langchain_openai import OpenAIEmbeddings  # Importación actualizada
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

# Definición de documentos de ejemplo
ejemplo_documentos_rag = [
    Document(
        page_content="""
        La probabilidad condicional es la probabilidad de que ocurra un evento A,
        sabiendo que también ha ocurrido otro evento B. Se denota como P(A|B) y
        se calcula como: P(A|B) = P(A∩B) / P(B)

        La regla de la cadena establece que: P(A∩B) = P(A|B) × P(B)
        """
    ),
    Document(
        page_content="""
        El Teorema de Bayes es una herramienta fundamental que relaciona las
        probabilidades condicionales de dos eventos. Su fórmula es:
        P(A|B) = P(B|A) × P(A) / P(B)

        Este teorema es especialmente útil cuando tenemos probabilidades previas
        y queremos actualizar nuestro conocimiento con nueva información.
        """
    ),
    Document(
        page_content="""
        Los árboles de probabilidad son herramientas visuales que ayudan a
        resolver problemas de probabilidad condicional. Cada rama representa
        un evento y sus probabilidades asociadas.
        """
    )
]

# Definición del Estado
class State(TypedDict):
    user_input: str
    modo: str
    respuestas: List[str]
    pregunta_idx: int
    feedback: Dict[str, Any]
    nivel: str
    fortalezas: List[str]
    debilidades: List[str]
    puntaje_promedio: float
    detalle: List[Dict[str, Any]]
    preguntas_seleccionadas: List[Dict[str, Any]]
    temas: List[str]
    subtemas: Dict[str, Any]
    tema_actual: int
    contexto_recuperado: str
    explicacion: str

# Configuración de componentes
embeddings = OpenAIEmbeddings()

# Crear y configurar Chroma
vectorstore = Chroma.from_documents(
    documents=ejemplo_documentos_rag,
    embedding=embeddings,
    collection_name="estadistica_probabilidad"
)

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)
llm = ChatOpenAI(model="o4-mini")
# Prompts
PROMPT_EXPLICACION = ChatPromptTemplate.from_messages([
    ("system", """Eres un tutor experto en estadística y probabilidad.
    Tu tarea es explicar conceptos de manera clara y estructurada usando formato markdown.
    Incluye definiciones, fórmulas y ejemplos prácticos."""),
    ("user", """Basándote en el siguiente contexto:
    {contexto}

    Genera una explicación en formato markdown del tema "{tema_actual}"
    cubriendo los siguientes subtemas:
    {subtemas_str}

    La explicación debe incluir:
    - Definiciones claras
    - Fórmulas cuando sea necesario
    - Ejemplos prácticos
    - Formato markdown bien estructurado
    """)
])

def recuperar_contexto(state: State) -> State:
    """Recupera información relevante del tema actual usando RAG"""
    tema_idx = state["tema_actual"]
    tema_actual = state["temas"][tema_idx]
    subtemas = state["subtemas"][tema_actual]

    # Construir query para RAG
    subtemas_str = "\n".join([f"- {s}" for s in subtemas])
    query = f"""
    Información detallada sobre {tema_actual}:
    {subtemas_str}
    Incluir definiciones, fórmulas y ejemplos.
    """

    # Obtener documentos relevantes
    documentos = retriever.get_relevant_documents(query)
    contexto = "\n\n".join([doc.page_content for doc in documentos])

    return {**state, "contexto_recuperado": contexto}

def generar_explicacion(state: State) -> State:
    """Genera la explicación en formato markdown"""
    tema_idx = state["tema_actual"]
    tema_actual = state["temas"][tema_idx]
    subtemas = state["subtemas"][tema_actual]
    contexto = state["contexto_recuperado"]

    # Preparar subtemas para el prompt
    subtemas_str = "\n".join([f"- {s}" for s in subtemas])

    # Generar explicación usando el LLM
    messages = PROMPT_EXPLICACION.format_messages(
        contexto=contexto,
        tema_actual=tema_actual,
        subtemas_str=subtemas_str
    )

    explicacion = llm.invoke(messages).content

    return {**state, "explicacion": explicacion}

def validar_estado(state: State) -> str:
    """Determina si continuar o finalizar basado en el estado"""
    return END

def crear_grafo_agente():
    """Crea y configura el grafo del agente"""
    workflow = StateGraph(State)

    # Agregar nodos
    workflow.add_node("recuperar_contexto", recuperar_contexto)
    workflow.add_node("generar_explicacion", generar_explicacion)

    # Configurar el flujo
    workflow.set_entry_point("recuperar_contexto")
    workflow.add_edge("recuperar_contexto", "generar_explicacion")
    workflow.add_edge("generar_explicacion", END)

    return workflow.compile()

ModuleNotFoundError: No module named 'langchain_chroma'

In [4]:
# Configurar estado inicial
estado_inicial = {
    'user_input': '',
    'modo': 'guiado',
    'respuestas': [],
    'pregunta_idx': 0,
    'feedback': {},
    'nivel': 'intermedio',
    'fortalezas': [],
    'debilidades': ['probabilidad condicional', 'distribuciones', 'teorema de Bayes'],
    'puntaje_promedio': 0.0,
    'detalle': [],
    'preguntas_seleccionadas': [],
    'temas': [
        'Probabilidad Condicional',
        'Distribuciones de Probabilidad',
        'Teorema de Bayes'
    ],
    'subtemas': {
        'Probabilidad Condicional': [
            'Definición y fórmula de P(A|B)',
            'Regla de la multiplicación',
            'Teorema de la probabilidad total',
            'Ejemplos prácticos con árboles de probabilidad'
        ],
        'Distribuciones de Probabilidad': [
            'Distribuciones discretas: Binomial y Poisson',
            'Distribuciones continuas: Uniforme y Normal',
            'Función de densidad y función de distribución acumulada',
            'Cálculo de esperanza y varianza'
        ],
        'Teorema de Bayes': [
            'Derivación de la fórmula de Bayes',
            'Priori, verosimilitud y posteriori',
            'Aplicaciones en diagnósticos médicos',
            'Resolución de problemas complejos'
        ]
    },
    'tema_actual': 0,  # Comenzará con Probabilidad Condicional
    'contexto_recuperado': '',
    'explicacion': ''
}

# Crear y ejecutar el agente
agente = crear_grafo_agente()
resultado = agente.invoke(estado_inicial)

# Imprimir la explicación generada
print("Tema actual:", estado_inicial['temas'][estado_inicial['tema_actual']])
print("\nExplicación generada:")
print(resultado['explicacion'])

# Para cambiar al siguiente tema, actualizar tema_actual
estado_inicial['tema_actual'] = 1  # Cambia a Distribuciones de Probabilidad
resultado_siguiente = agente.invoke(estado_inicial)

Tema actual: Probabilidad Condicional

Explicación generada:
# Probabilidad Condicional

La probabilidad condicional es un concepto fundamental en teoría de la probabilidad que consiste en determinar la probabilidad de que ocurra un evento A, dado que sabemos que otro evento B ha ocurrido. Se denota como P(A|B) y se calcula dividiendo la probabilidad de la intersección de A y B entre la probabilidad de B.

## Definición y fórmula de P(A|B)

La probabilidad condicional de A dado B se define como:

\[
P(A|B) = \frac{P(A \cap B)}{P(B)}
\]

Donde:
- \(P(A|B)\) es la probabilidad de que ocurra A dado que ha ocurrido B.
- \(P(A \cap B)\) es la probabilidad de que ocurran ambos eventos A y B.
- \(P(B)\) es la probabilidad de que ocurra B.

## Regla de la multiplicación

La regla de la multiplicación se utiliza para calcular la probabilidad de la intersección de dos eventos. Esta regla es fundamental en la teoría de la probabilidad y se expresa de la siguiente manera:

\[
P(A \cap B) = P(A|B) 

In [6]:
print(resultado_siguiente['explicacion'])

# Distribuciones de Probabilidad

Las distribuciones de probabilidad son modelos matemáticos que describen la probabilidad de ocurrencia de cada resultado posible en un experimento aleatorio. Existen distribuciones de probabilidad discretas y continuas, cada una con sus propias características y aplicaciones.

## Distribuciones Discretas

### Distribución Binomial

La distribución binomial modela el número de éxitos en una secuencia de n ensayos independientes, donde cada ensayo tiene una probabilidad de éxito p y una probabilidad de fracaso q = 1 - p.

- **Fórmula de probabilidad:** $P(X=k) = \binom{n}{k} \cdot p^k \cdot q^{n-k}$
- **Esperanza:** $E(X) = n \cdot p$
- **Varianza:** $Var(X) = n \cdot p \cdot q$

**Ejemplo:** En un lanzamiento de una moneda justa (p = 0.5) 5 veces, ¿cuál es la probabilidad de obtener exactamente 3 caras?

$P(X=3) = \binom{5}{3} \cdot 0.5^3 \cdot 0.5^2 = 10 \cdot 0.125 \cdot 0.25 = 0.3125$

### Distribución de Poisson

La distribución de Poisson se utiliz

# Nodo respuestas (RAG)

# Tools